# Syqure Python Demo

This notebook demonstrates using the syqure Python library to compile and run Codon/Sequre programs.

Syqure is a Rust wrapper around Codon/Sequre that:
- Bundles all required libraries (Codon runtime, Sequre plugin)
- Auto-extracts to `~/.cache/syqure/` on first use
- Provides both CLI and Python API

**Note:** If you've just updated the syqure package, restart the Jupyter kernel before running these cells.

## 1. Build the Python Wheel

First, build the syqure Python package from source:

In [1]:
# Build the wheel (uses existing bundle, skips rebuilding libs)
# Uncomment to run:
!cd .. && bash build_python.sh --skip-libs

==> Using existing bundle: /Users/madhavajay/dev/syqure/main/syqure/bundles/aarch64-apple-darwin.tar.zst
==> Building release wheel
🍹 Building a mixed python/rust project
🔗 Found pyo3 bindings with abi3 support
🐍 Found CPython 3.13 at /Users/madhavajay/dev/syqure/main/.venv-maturin/bin/python
📡 Using build options features from pyproject.toml
💻 Using `MACOSX_DEPLOYMENT_TARGET=11.0` for aarch64-apple-darwin by default
   Compiling libc v0.2.177
   Compiling proc-macro2 v1.0.103
   Compiling quote v1.0.42
   Compiling unicode-ident v1.0.22
   Compiling shlex v1.3.0
   Compiling find-msvc-tools v0.1.5
   Compiling target-lexicon v0.12.16
   Compiling once_cell v1.21.3
   Compiling pkg-config v0.3.32
   Compiling hashbrown v0.16.1
   Compiling equivalent v1.0.2
   Compiling scratch v1.0.9
   Compiling cfg-if v1.0.4               ] 0/132: quote(build.rs), unicode-...
   Compiling cxxbridge-flags v1.0.189    ] 1/132: quote(build.rs), unicode-...
   Compiling memchr v2.7.6            ] 2/132:

## 2. Install the Wheel

Install the built wheel into the current environment:

In [2]:
# Find and install the latest wheel
# Uncomment to run:
import glob, os
wheels = sorted(glob.glob('../target/wheels/syqure-*.whl'), key=os.path.getmtime, reverse=True)
if wheels:
    !pip install --force-reinstall "{wheels[0]}"
else:
    print("No wheel found. Run the build cell first.")

Processing /Users/madhavajay/dev/syqure/main/target/wheels/syqure-0.1.21-cp38-abi3-macosx_11_0_arm64.whl
  Attempting uninstall: syqure
    Found existing installation: syqure 0.1.21
    Uninstalling syqure-0.1.21:
      Successfully uninstalled syqure-0.1.21


## 3. Import and Check Version

In [3]:
import syqure

print(f"Syqure version: {syqure.version()}")
print(f"Module: {syqure.__doc__}")

Syqure version: 0.1.21
Module: Python bindings for the syqure Rust library - Codon/Sequre compiler wrapper using PyO3.


## 4. System Information

Check syqure's configuration and bundled libraries:

In [4]:
import pprint

info = syqure.info()
pprint.pprint(info)

{'arch': 'aarch64',
 'cache_dir': '/Users/madhavajay/.cache/syqure/aarch64-apple-darwin.tar.zst/2db44c21ecd748c4-170894156/lib',
 'codon_path': '/Users/madhavajay/.cache/syqure/aarch64-apple-darwin.tar.zst/2db44c21ecd748c4-170894156/lib/codon',
 'environment': {},
 'libraries': ['libgmp.dylib',
               'libcodonrt.dylib',
               'libomp.dylib',
               'libcodonc.dylib',
               '._libgmp.dylib',
               '._libcodonrt.dylib',
               '._libcodon_jupyter.dylib',
               '._libomp.dylib',
               'libgmp.so',
               '._libgmp.so',
               'libcodon_jupyter.dylib',
               '._libcodonc.dylib'],
 'os': 'macos',
 'plugins': ['sequre'],
 'profile': 'debug',
 'target': 'aarch64-apple-darwin',
 'version': '0.1.21'}


## 5. Hello World Example

Create and run a simple Codon program:

In [5]:
# Create a simple test file
test_code = '''
# Simple Codon program
print("Hello from Codon!")
print(f"2 + 2 = {2 + 2}")

# Codon supports Python-like syntax with native performance
def fibonacci(n: int) -> int:
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(f"fibonacci(10) = {fibonacci(10)}")
'''

# Write to temp file
with open('/tmp/hello.codon', 'w') as f:
    f.write(test_code)

print("Created /tmp/hello.codon")
print("---")
print(test_code)

Created /tmp/hello.codon
---

# Simple Codon program
print("Hello from Codon!")
print(f"2 + 2 = {2 + 2}")

# Codon supports Python-like syntax with native performance
def fibonacci(n: int) -> int:
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(f"fibonacci(10) = {fibonacci(10)}")



In [6]:
# Run the program - output is captured and displayed in Python
syqure.compile_and_run('/tmp/hello.codon')

Hello from Codon!
2 + 2 = 4
fibonacci(10) = 55



## 6. Analyze a Program

Use `syqure.analyze()` to get cost estimates and detect MPC/HE usage:

In [7]:
# Analyze the hello world program
analysis = syqure.analyze('/tmp/hello.codon')
pprint.pprint(analysis)

{'estimate': {'jit_seconds': 20.0,
              'mhe_seconds': 0.0,
              'mpc_seconds': 3.0,
              'ops_seconds': 0.0,
              'total_seconds': 23.0},
 'file': 'hello.codon',
 'operations': {'decrypt': 0, 'encrypt': 0, 'matmul': 0},
 'path': '/private/tmp/hello.codon',
 'runtime': {'can_skip_mhe': True, 'needs_mhe': False, 'uses_local': False},
 'types': {'ciphertensor': False,
           'mpa': False,
           'mpp': False,
           'mpu': False,
           'sharetensor': False}}


## 7. Using CompileOptions

Customize compilation with `CompileOptions`:

In [8]:
# Run with release mode optimization
opts = syqure.CompileOptions(
    release=True,           # Optimize for performance
    run_after_build=True,   # Run after compilation
)

# Compile and run with options
syqure.compile_and_run('/tmp/hello.codon', opts)

Hello from Codon!
2 + 2 = 4
fibonacci(10) = 55



## 8. Using the Syqure Class

For repeated compilations, use the `Syqure` class:

In [9]:
# Create a compiler instance with default options
compiler = syqure.Syqure.default()

# Or with custom options
opts = syqure.CompileOptions(release=False, run_after_build=True)
compiler = syqure.Syqure(opts)

# Use it to compile and run (use hello.codon, not MPC which requires network setup)
compiler.compile_and_run('/tmp/hello.codon')

Hello from Codon!
2 + 2 = 4
fibonacci(10) = 55



## 9. MPC Example Analysis

Analyze an MPC program to understand its requirements. The `@local` decorator enables local simulation by forking multiple processes.

**Note:** Running MPC programs via Python API may crash due to process forking conflicts with output capture. Use the CLI for running MPC programs:

In [ ]:
# Use the two_party_sum_simple example (works locally with @local decorator)
import shutil
shutil.copy('../example/two_party_sum_simple.codon', '/tmp/two_party_sum.codon')

# Show the code
with open('/tmp/two_party_sum.codon') as f:
    print(f.read())

In [ ]:
# Analyze the MPC program
analysis = syqure.analyze('/tmp/two_party_sum.codon')

print("--- Analysis ---")
print(f"Uses Sharetensor (MPC): {analysis['types']['sharetensor']}")
print(f"Uses @local decorator: {analysis['runtime']['uses_local']}")
print(f"Requires MHE setup: {analysis['runtime']['needs_mhe']}")
print(f"Can use --skip-mhe-setup: {analysis['runtime']['can_skip_mhe']}")
print(f"Decrypt/reveal calls: {analysis['operations']['decrypt']}")
print(f"Estimated total time: {analysis['estimate']['total_seconds']}s")

In [ ]:
# Run MPC example via CLI (Python API crashes due to process forking in @local)
# Build CLI first if needed: cd ../syqure && cargo build --release
!../target/release/syqure /tmp/two_party_sum.codon --skip-mhe-setup

In [ ]:
# Create a compiler instance with default options
compiler = syqure.Syqure.default()

# Or with custom options
opts = syqure.CompileOptions(release=False, run_after_build=True)
compiler = syqure.Syqure(opts)

# Use it to compile and run
compiler.compile_and_run('/tmp/mpc_sum.codon')

## API Summary

### Python Functions

| Function | Description |
|----------|-------------|
| `syqure.version()` | Returns version string |
| `syqure.info()` | Returns dict with build/system info |
| `syqure.analyze(path)` | Analyze file for cost estimation |
| `syqure.compile_and_run(path, opts?)` | Compile and run file |

### CompileOptions

```python
syqure.CompileOptions(
    codon_path=None,      # Override Codon path
    plugin=None,          # Plugin name (e.g., 'sequre')
    release=False,        # Release mode optimization
    run_after_build=True, # Must be True (build-only not yet supported)
    program_args=None,    # Args to pass to program (e.g., ['--skip-mhe-setup'])
)
```

### Notes

- **Output capture**: The Python API captures stdout/stderr from compiled Codon programs and writes them to Python's stdout/stderr, so output appears in Jupyter notebook cells.
- **MPC programs**: Programs using `@local` decorator (which forks processes) may crash when run via Python API. Use the CLI for MPC programs.
- **Build-only mode**: `run_after_build=False` is not currently supported as the bundle doesn't include the codon linker.